***GENERATED CODE FOR dp25oct PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        #hdfs = HDFileSystem(host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'customerID', 'transformation_label': 'String Indexer'}], 'feature': 'customerID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '0021-IKXGC', 'max': '9967-ATRFS', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'customerID'}, {'feature_label': 'customerID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('customerID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'gender', 'transformation_label': 'String Indexer'}], 'feature': 'gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'gender'}, {'feature_label': 'gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Partner', 'transformation_label': 'String Indexer'}], 'feature': 'Partner', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Partner'}, {'feature_label': 'Partner', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Partner')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Dependents', 'transformation_label': 'String Indexer'}], 'feature': 'Dependents', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Dependents'}, {'feature_label': 'Dependents', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Dependents')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PhoneService', 'transformation_label': 'String Indexer'}], 'feature': 'PhoneService', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PhoneService'}, {'feature_label': 'PhoneService', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PhoneService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MultipleLines', 'transformation_label': 'String Indexer'}], 'feature': 'MultipleLines', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MultipleLines'}, {'feature_label': 'MultipleLines', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MultipleLines')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternetService', 'transformation_label': 'String Indexer'}], 'feature': 'InternetService', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'DSL', 'max': 'No', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternetService'}, {'feature_label': 'InternetService', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternetService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OnlineSecurity', 'transformation_label': 'String Indexer'}], 'feature': 'OnlineSecurity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OnlineSecurity'}, {'feature_label': 'OnlineSecurity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OnlineSecurity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OnlineBackup', 'transformation_label': 'String Indexer'}], 'feature': 'OnlineBackup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OnlineBackup'}, {'feature_label': 'OnlineBackup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OnlineBackup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DeviceProtection', 'transformation_label': 'String Indexer'}], 'feature': 'DeviceProtection', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DeviceProtection'}, {'feature_label': 'DeviceProtection', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DeviceProtection')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TechSupport', 'transformation_label': 'String Indexer'}], 'feature': 'TechSupport', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TechSupport'}, {'feature_label': 'TechSupport', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TechSupport')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StreamingTV', 'transformation_label': 'String Indexer'}], 'feature': 'StreamingTV', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StreamingTV'}, {'feature_label': 'StreamingTV', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StreamingTV')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StreamingMovies', 'transformation_label': 'String Indexer'}], 'feature': 'StreamingMovies', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StreamingMovies'}, {'feature_label': 'StreamingMovies', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StreamingMovies')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Contract', 'transformation_label': 'String Indexer'}], 'feature': 'Contract', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Month-to-month', 'max': 'Two year', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Contract'}, {'feature_label': 'Contract', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Contract')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PaperlessBilling', 'transformation_label': 'String Indexer'}], 'feature': 'PaperlessBilling', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PaperlessBilling'}, {'feature_label': 'PaperlessBilling', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PaperlessBilling')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PaymentMethod', 'transformation_label': 'String Indexer'}], 'feature': 'PaymentMethod', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Bank transfer (automatic)', 'max': 'Mailed check', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PaymentMethod'}, {'feature_label': 'PaymentMethod', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PaymentMethod')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Churn', 'transformation_label': 'String Indexer'}], 'feature': 'Churn', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Churn'}, {'feature_label': 'Churn', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Churn')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run dp25octHooks.ipynb
try:
	#sourcePreExecutionHook()

	telcocutomerchurntrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1692339300908_Telco_Customer_Churn_Train.csv', 'filename': 'Telco_Customer_Churn_Train.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Retail Marketing/Telco Churn/Telco_Customer_Churn_Train.csv', 'viewFileName': 'Telco_Customer_Churn_Train.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run dp25octHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(telcocutomerchurntrain,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "customerID", "transformation_label": "String Indexer"}], "feature": "customerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "0021-IKXGC", "max": "9967-ATRFS", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "customerID"}, {"transformationsData": [{"feature_label": "gender", "transformation_label": "String Indexer"}], "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SeniorCitizen", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.17", "stddev": "0.37", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "SeniorCitizen"}, {"transformationsData": [{"feature_label": "Partner", "transformation_label": "String Indexer"}], "feature": "Partner", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Partner"}, {"transformationsData": [{"feature_label": "Dependents", "transformation_label": "String Indexer"}], "feature": "Dependents", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Dependents"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "tenure", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "31.45", "stddev": "24.42", "min": "0", "max": "72", "missing": "0"}, "updatedLabel": "tenure"}, {"transformationsData": [{"feature_label": "PhoneService", "transformation_label": "String Indexer"}], "feature": "PhoneService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PhoneService"}, {"transformationsData": [{"feature_label": "MultipleLines", "transformation_label": "String Indexer"}], "feature": "MultipleLines", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MultipleLines"}, {"transformationsData": [{"feature_label": "InternetService", "transformation_label": "String Indexer"}], "feature": "InternetService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "DSL", "max": "No", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternetService"}, {"transformationsData": [{"feature_label": "OnlineSecurity", "transformation_label": "String Indexer"}], "feature": "OnlineSecurity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OnlineSecurity"}, {"transformationsData": [{"feature_label": "OnlineBackup", "transformation_label": "String Indexer"}], "feature": "OnlineBackup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OnlineBackup"}, {"transformationsData": [{"feature_label": "DeviceProtection", "transformation_label": "String Indexer"}], "feature": "DeviceProtection", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DeviceProtection"}, {"transformationsData": [{"feature_label": "TechSupport", "transformation_label": "String Indexer"}], "feature": "TechSupport", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TechSupport"}, {"transformationsData": [{"feature_label": "StreamingTV", "transformation_label": "String Indexer"}], "feature": "StreamingTV", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StreamingTV"}, {"transformationsData": [{"feature_label": "StreamingMovies", "transformation_label": "String Indexer"}], "feature": "StreamingMovies", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StreamingMovies"}, {"transformationsData": [{"feature_label": "Contract", "transformation_label": "String Indexer"}], "feature": "Contract", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Month-to-month", "max": "Two year", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Contract"}, {"transformationsData": [{"feature_label": "PaperlessBilling", "transformation_label": "String Indexer"}], "feature": "PaperlessBilling", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PaperlessBilling"}, {"transformationsData": [{"feature_label": "PaymentMethod", "transformation_label": "String Indexer"}], "feature": "PaymentMethod", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Bank transfer (automatic)", "max": "Mailed check", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PaymentMethod"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MonthlyCharges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "65.97", "stddev": "29.55", "min": "18.95", "max": "116.05", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MonthlyCharges"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TotalCharges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "2223.41", "stddev": "2198.76", "min": "19.15", "max": "8468.2", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TotalCharges"}, {"transformationsData": [{"feature_label": "Churn", "transformation_label": "String Indexer"}], "feature": "Churn", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Churn"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run dp25octHooks.ipynb
try:
	#sinkPreExecutionHook()

	test = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/65390206c41e2f1a07177c0a/part-00000-57e4a7e9-668f-4fa7-8fb2-94df0c2d2cea-c000.csv', 'samplecount': 741, 'originalcount': 7043, 'orignalKey': '/FileStore/platform/testdata/1692339300908_Telco_Customer_Churn_Train.csv', 'pathOnly': '/test', 'project_id': '65114a045419cf046e2dfd2b', 'parent_id': '65114a045419cf046e2dfd2b', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c62', 'field': 'SeniorCitizen', 'alias': 'SeniorCitizen', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c63', 'field': 'tenure', 'alias': 'tenure', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c64', 'field': 'MonthlyCharges', 'alias': 'MonthlyCharges', 'type': 'real', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c65', 'field': 'TotalCharges', 'alias': 'TotalCharges', 'type': 'real', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c66', 'field': 'customerID_stringindexer', 'alias': 'customerID_stringindexer', 'type': 'real', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c67', 'field': 'gender_stringindexer', 'alias': 'gender_stringindexer', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c68', 'field': 'Partner_stringindexer', 'alias': 'Partner_stringindexer', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c69', 'field': 'Dependents_stringindexer', 'alias': 'Dependents_stringindexer', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c6a', 'field': 'PhoneService_stringindexer', 'alias': 'PhoneService_stringindexer', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c6b', 'field': 'MultipleLines_stringindexer', 'alias': 'MultipleLines_stringindexer', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c6c', 'field': 'InternetService_stringindexer', 'alias': 'InternetService_stringindexer', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c6d', 'field': 'OnlineSecurity_stringindexer', 'alias': 'OnlineSecurity_stringindexer', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c6e', 'field': 'OnlineBackup_stringindexer', 'alias': 'OnlineBackup_stringindexer', 'type': 'numeric', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c6f', 'field': 'DeviceProtection_stringindexer', 'alias': 'DeviceProtection_stringindexer', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c70', 'field': 'TechSupport_stringindexer', 'alias': 'TechSupport_stringindexer', 'type': 'numeric', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c71', 'field': 'StreamingTV_stringindexer', 'alias': 'StreamingTV_stringindexer', 'type': 'numeric', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c72', 'field': 'StreamingMovies_stringindexer', 'alias': 'StreamingMovies_stringindexer', 'type': 'numeric', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c73', 'field': 'Contract_stringindexer', 'alias': 'Contract_stringindexer', 'type': 'numeric', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c74', 'field': 'PaperlessBilling_stringindexer', 'alias': 'PaperlessBilling_stringindexer', 'type': 'numeric', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c75', 'field': 'PaymentMethod_stringindexer', 'alias': 'PaymentMethod_stringindexer', 'type': 'numeric', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c76', 'field': 'Churn_stringindexer', 'alias': 'Churn_stringindexer', 'type': 'numeric', 'position': '20'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c4d', 'field': 'customerID', 'alias': 'customerID', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c4e', 'field': 'gender', 'alias': 'gender', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c4f', 'field': 'SeniorCitizen', 'alias': 'SeniorCitizen', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c50', 'field': 'Partner', 'alias': 'Partner', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c51', 'field': 'Dependents', 'alias': 'Dependents', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c52', 'field': 'tenure', 'alias': 'tenure', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c53', 'field': 'PhoneService', 'alias': 'PhoneService', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c54', 'field': 'MultipleLines', 'alias': 'MultipleLines', 'type': 'String', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c55', 'field': 'InternetService', 'alias': 'InternetService', 'type': 'String', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c56', 'field': 'OnlineSecurity', 'alias': 'OnlineSecurity', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c57', 'field': 'OnlineBackup', 'alias': 'OnlineBackup', 'type': 'String', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c58', 'field': 'DeviceProtection', 'alias': 'DeviceProtection', 'type': 'String', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c59', 'field': 'TechSupport', 'alias': 'TechSupport', 'type': 'String', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c5a', 'field': 'StreamingTV', 'alias': 'StreamingTV', 'type': 'String', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c5b', 'field': 'StreamingMovies', 'alias': 'StreamingMovies', 'type': 'String', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c5c', 'field': 'Contract', 'alias': 'Contract', 'type': 'String', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c5d', 'field': 'PaperlessBilling', 'alias': 'PaperlessBilling', 'type': 'String', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c5e', 'field': 'PaymentMethod', 'alias': 'PaymentMethod', 'type': 'String', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c5f', 'field': 'MonthlyCharges', 'alias': 'MonthlyCharges', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c60', 'field': 'TotalCharges', 'alias': 'TotalCharges', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '653902a9c41e2f1a07177c61', 'field': 'Churn', 'alias': 'Churn', 'type': 'String', 'position': '20'}], 'server': 'https://ml.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': '25octtest.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
